### Author - Saurabh Mangal

# Installation

### Speaker Diaritization 

In [1]:
# !sudo apt-get install -y wkhtmltopdf                                                   # Installing wkhtmltopdf command-line tools.
!pip install google-cloud-aiplatform google-cloud-speech --upgrade --quiet --user      # Installing Google Cloud SDK and Cloud Speech SDK
!pip install PyPDF2 pdfkit --upgrade --quiet --user                    # Installing PDF handling tools
# !pip install PyPDF2 py3-wkhtmltopdf pdfkit --upgrade --quiet --user                    # Installing PDF handling tools

!pip install ratelimit backoff --upgrade --quiet --user    

In [4]:
# !sudo apt-get install -y wkhtmltopdf     
## Installing wkhtmltopdf command-line tools.
# !pip install google-cloud-aiplatform google-cloud-speech --upgrade --quiet --user      # Installing Google Cloud SDK and Cloud Speech SDK
# !pip install PyPDF2 py3-wkhtmltopdf pdfkit --upgrade --quiet --user                    # Installing PDF handling tools
# !pip install ratelimit backoff --upgrade --quiet --user   

In [11]:
### This it the change -- USER  may have to do -- ##

VERTEX_API_PROJECT = PROJECT_ID = "my-project-0004-346516" #'your-project' #@param {"type": "string"}
VERTEX_API_LOCATION =REGION= 'us-central1' #@param {"type": "string"}



### Simple Speech to text using V1 with Diaratization

In [60]:
# !ffmpeg -i podcast_mu.m4a podcast_out.wav

# !ffmpeg -i podcast_mu.m4a -c:a aac -b:a 256k podcast_out2.aac

# !ffmpeg -i podcast_mu.m4a -ss 00:00:05 -t 00:00:15 podcast_out4.wav


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [62]:
from google.cloud import speech_v1p1beta1 as speech

client = speech.SpeechClient()

speech_file = "./podcast_out4.wav"  ### This it the change -- USER  may have to do -- ##

with open(speech_file, "rb") as audio_file:
    content = audio_file.read()

audio = speech.RecognitionAudio(content=content)

diarization_config = speech.SpeakerDiarizationConfig(
    enable_speaker_diarization=True,
    min_speaker_count=2,
    max_speaker_count=10,
)

config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    # sample_rate_hertz=8000,
    sample_rate_hertz=44100,
    audio_channel_count = 2,
    language_code="id-ID",
    diarization_config=diarization_config,
)

print("Waiting for operation to complete...")
response = client.recognize(config=config, audio=audio)

# The transcript within each result is separate and sequential per result.
# However, the words list within an alternative includes all the words
# from all the results thus far. Thus, to get all the words with speaker
# tags, you only have to take the words list from the last result:
result = response.results[-1]

words_info = result.alternatives[0].words

# Printing out the output:
for word_info in words_info:
    print(f"word: '{word_info.word}', speaker_tag: {word_info.speaker_tag}")
# return result


print("Building transcriptions...")
transcript_builder = []
# Each result is for a consecutive portion of the audio. Iterate through
# them to get the transcripts for the entire audio file.
for result in response.results:
    alternative = result.alternatives[0]

    # The first alternative is the most likely one for this portion.
    transcript_builder.append(f"\n{result.alternatives[0].transcript}")
    # transcript_builder.append(f"\nConfidence: {result.alternatives[0].confidence}")

transcript = ''.join(transcript_builder)
print(transcript)

Waiting for operation to complete...
word: 'opening', speaker_tag: 1
word: 'soal', speaker_tag: 1
word: 'mu', speaker_tag: 1
word: 'aja', speaker_tag: 1
word: 'sering', speaker_tag: 1
word: 'kita', speaker_tag: 1
word: 'bosen', speaker_tag: 1
word: 'itu', speaker_tag: 1
word: 'kita', speaker_tag: 1
word: 'butuh', speaker_tag: 1
word: 'topik', speaker_tag: 1
word: 'lain', speaker_tag: 1
word: 'Hasilnya', speaker_tag: 1
word: 'kayak', speaker_tag: 1
word: 'begini', speaker_tag: 1
word: 'masalah', speaker_tag: 1
word: 'dibahas', speaker_tag: 1
word: 'Apa', speaker_tag: 1
word: 'perlu', speaker_tag: 1
word: 'gue', speaker_tag: 1
word: 'doain', speaker_tag: 1
word: 'mu', speaker_tag: 1
word: 'supaya', speaker_tag: 1
word: 'menang', speaker_tag: 1
word: 'terus', speaker_tag: 1
Building transcriptions...

opening soal mu aja sering kita bosen itu kita butuh topik lain Hasilnya kayak begini masalah dibahas Apa perlu gue doain mu supaya menang terus



## Speech to text using V1 Summary

In [38]:
from google.cloud import speech_v1p1beta1 as speech

client = speech.SpeechClient()

# speech_file = "resources/commercial_mono.wav"

with open(speech_file, "rb") as audio_file:
    content = audio_file.read()

audio = speech.RecognitionAudio(content=content)
config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    # sample_rate_hertz=8000,
    sample_rate_hertz=44100,
    audio_channel_count = 2,
    language_code="id-ID", #"id-ID" en-US
    # use_enhanced=True,
    # A model must be specified to use enhanced model.
    # model="long",
)

response = client.recognize(config=config, audio=audio)

for i, result in enumerate(response.results):
    alternative = result.alternatives[0]
    print("-" * 20)
    print(f"First alternative of result {i}")
    print(f"Transcript: {alternative.transcript}")
    fina_response = alternative.transcript

print("Final response ::: ",fina_response)
# return response.results

--------------------
First alternative of result 0
Transcript: the still smile of Wonders Water World that we should take time to the Bone
Final response :::  the still smile of Wonders Water World that we should take time to the Bone


In [43]:
!ls

'(SQL)_data_eval_[NTK_Do_not_share_Externally].ipynb'
'BigQuery One Pager.pdf'
'Cost optimization best practices for BigQuery.pdf'
 LLM_Evaluation_Demo_Notebook_Vertex_Evaluation_for_LLM.ipynb
 Speech_to_text.ipynb
'Untitled Folder'
 Untitled.ipynb
'[external]_reinforcement_learning_from_ai_feedback_tutorial.ipynb'
 applied-ai-engineering-samples
 bahasa-diarization
 embedding-similarity-visualization.ipynb
 google_cloud_aiplatform-1.26.dev20230530+language.models.eval-py2.py3-none-any.whl
 google_cloud_aiplatform-1.38.dev20231206+generative.models-py2.py3-none-any.whl
 harvard.wav
 podcast_mu.m4a
 podcast_out.wav
 rlhf_pipeline.yaml
 shard-00000-of-00002.jsonl
 shard-00001-of-00002.jsonl
 stepback-qa.ipynb
 test_table_pdf.csv
 test_table_pdf.pdf


In [63]:
from google.cloud import speech_v1p1beta1 as speech

client = speech.SpeechClient()

# speech_file = "./harvard.wav"
speech_file = './podcast_out3.wav'

with open(speech_file, "rb") as audio_file:
    content = audio_file.read()

audio = speech.RecognitionAudio(content=content)

diarization_config = speech.SpeakerDiarizationConfig(
    enable_speaker_diarization=True,
    min_speaker_count=2,
    max_speaker_count=10,
)

config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    # sample_rate_hertz=8000,
    sample_rate_hertz=44100,
    audio_channel_count = 2,
    language_code="id-ID",
    model="latest_long",
    diarization_config=diarization_config,
)

print("Waiting for operation to complete...")
response = client.recognize(config=config, audio=audio)

# The transcript within each result is separate and sequential per result.
# However, the words list within an alternative includes all the words
# from all the results thus far. Thus, to get all the words with speaker
# tags, you only have to take the words list from the last result:
result = response.results[-1]

words_info = result.alternatives[0].words

# Printing out the output:
for word_info in words_info:
    print(f"word: '{word_info.word}', speaker_tag: {word_info.speaker_tag}")
# return result

Waiting for operation to complete...
word: 'jangankan', speaker_tag: 1
word: 'lo', speaker_tag: 1
word: 'yang', speaker_tag: 1
word: 'nonton', speaker_tag: 1
word: 'dan', speaker_tag: 1
word: 'bilang', speaker_tag: 1
word: 'kita', speaker_tag: 1
word: 'podcast', speaker_tag: 1
word: 'mu', speaker_tag: 1
word: 'bahasa', speaker_tag: 1
word: 'mu', speaker_tag: 1
word: 'terus', speaker_tag: 1


## Speech to text using V2 latest

In [54]:
from google.cloud.speech_v2 import SpeechClient
from google.cloud.speech_v2.types import cloud_speech


In [57]:
def quickstart_v2(
    project_id: str,
    audio_file: str,
) -> cloud_speech.RecognizeResponse:
    """Transcribe an audio file."""
    # Instantiates a client
    client = SpeechClient()

    # Reads a file as bytes
    with open(audio_file, "rb") as f:
        content = f.read()

    config = cloud_speech.RecognitionConfig(
        auto_decoding_config=cloud_speech.AutoDetectDecodingConfig(),
        language_codes=["id-ID"],
        model="long",
    )

    request = cloud_speech.RecognizeRequest(
        recognizer=f"projects/{project_id}/locations/global/recognizers/_",
        config=config,
        content=content,
    )

    # Transcribes the audio into text
    response = client.recognize(request=request)

    for result in response.results:
        print(f"Transcript: {result.alternatives[0].transcript}")

    return response

In [58]:
quickstart_v2(PROJECT_ID,speech_file)

Transcript: jangankan lo yang nonton dan bilang kita podcast mu bahasa mu terus


results {
  alternatives {
    transcript: "jangankan lo yang nonton dan bilang kita podcast mu bahasa mu terus"
    confidence: 0.904364884
  }
  result_end_offset {
    seconds: 4
    nanos: 920000000
  }
  language_code: "id-ID"
}
metadata {
  total_billed_duration {
    seconds: 5
  }
}

## USING V2 and Chrip model (need to create a recognizer "bahasa-test" 

In [ ]:
GCS_Bucket = "genai-experiments-sm" #@param {type:"string"}
FILENAME = "podcast_out3.wav" #@param {type:"string"}
# This section is a WIP, incorporate v2 for chirp (this limits times to <60 seconds)

from google.cloud.speech_v2 import SpeechClient
from google.cloud.speech_v2.types import cloud_speech

gcs_uri = "gs://"+GCS_Bucket+"/"+FILENAME

def transcribe_gcs_v2(
    project_id: str,
    recognizer_id: str,
    gcs_uri: str,
) -> cloud_speech.RecognizeResponse:
    """Transcribes audio from a Google Cloud Storage URI.

    Args:
        project_id: The GCP project ID.
        recognizer_id: The ID of the recognizer.
        gcs_uri: The Google Cloud Storage URI.

    Returns:
        The RecognizeResponse.
    """
    # Instantiates a client
    client = SpeechClient()

    request = cloud_speech.CreateRecognizerRequest(
        parent=f"projects/{project_id}/locations/global",
        # parent=f"projects/{project_id}/locations/asia-southeast1",
        recognizer_id=recognizer_id,
        recognizer=cloud_speech.Recognizer(
            language_codes=["id-ID"], model="chirp"
        ),
    )

    # Creates a Recognizer
    operation = client.create_recognizer(request=request)
    recognizer = operation.result()

    config = cloud_speech.RecognitionConfig(auto_decoding_config={})

    request = cloud_speech.RecognizeRequest(
        recognizer=recognizer.name, config=config, uri=gcs_uri
    )

    # Transcribes the audio into text
    response = client.recognize(request=request)

    for result in response.results:
        print(f"Transcript: {result.alternatives[0].transcript}")

    return response

# response = transcribe_gcs_v2("icm-testing-env","testing123123123321",gcs_uri)

response = transcribe_gcs_v2(PROJECT_ID,"bahasa-test2",speech_file)

In [26]:
# print(response)

### USing Vertex AI

In [ ]:
# from langchain.chat_models import VertexAI
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

from langchain.llms import VertexAI

In [ ]:
# PROJECT_ID = "YOUR_PROJECT_ID_HERE"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
# Code examples may misbehave if the model is changed.
MODEL_NAME = "text-bison@001"

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at summarizing conversations. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        #few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)

from langchain.llms import VertexAI

# First, let's load the language model we're going to use to control the agent.
# llm = VertexAI(temperature=0.2)
# llm = VertexAI(temperature=0.2,model_name='text-bison@001')

llm = VertexAI(temperature=0.2,model_name='text-bison@latest')

model = model_name = 'text-bison@latest'

In [ ]:
def call_llm(model, parameters, llm_call, show_activity = True):
  response = model.predict(llm_call, **parameters).text

  if show_activity:
    BOLD = "\033[1m"
    UNFORMAT = "\033[0m\x1B[0m"
    print(f"{BOLD}The call to the LLM:{UNFORMAT}\n{llm_call}\n")
    print(f"{BOLD}The response:{UNFORMAT}")
    print(response)

  return response  # Return to `_` if not needed.


# Wrap code cell output to improve notebook readability.
# Source: https://stackoverflow.com/questions/58890109/line-wrapping-in-collaboratory-google-results/61401455#61401455
from IPython.display import HTML, display

# def set_css():
#   display(HTML('''
#   <style>
#     pre {
#         white-space: pre-wrap;
#     }
#   </style>
#   '''))
# get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
question = """ Conversation """ + transcript + """
summarize the convesation but combing the speaker tags 
A:"""

_ = call_llm(model, parameters, question)

In [ ]:
question_gen.invoke({"question": question})

### Actions :-- 


In [ ]:

## Trascribe bahasa
## Compare accuracy 
## Using Gen AI - Summarize , extract insights 


### NOtes

https://cloud.google.com/speech-to-text/#

https://console.cloud.google.com/vertex-ai/generative/speech/speech-to-text?hl=en_US
